In [1]:
import sys

sys.path.append("lib")

from functorch import vmap

import matplotlib.pyplot as plt

import lib
import torch
import math

In [2]:
Nx = 10
Ny = 10
mesh = lib.mesh.RectangleMesh({"xmin": 0, "xmax": 10, "ymin": 0, "ymax": 10, "Nx": Nx, "Ny": Ny})

problem = lib.problem.FEProblem(mesh)
problem.add_variable("ux", lambda x : torch.sin(x[0]))
problem.add_variable("uy", lambda x : torch.sin(x[1]))
problem.init_solution()
problem.write_vtk("foo.vtk")